# Cleaning the Chicago Police Use-Of-Force Citizen Complaints Data
#### Mihir Bhaskar
#### 11/23/2021

The following file reads in a raw .csv data file on citizen complaints of police use-of-force sourced from: https://data.cpdp.co/data/bVBkzB/ (accessed on 21st November, 2021). 

It performs the following tasks:
1. Cleans, processes, and merges all the subtables in the data (i.e. on complaints, police officers, and witnesses), keeping only complaints from 2015 onwards
2. Creates and outputs a basic dataset at the tract-level, called 'CleanComplaints', that has all tract IDs in Chicago along with the number of complaints from that tract. It does this by first doing a a spatial merge based on lat-long with the CleanACSFile data outputted from 2_CleanACS to get the tract ID for each complaint, and aggregates data up to the tract level. 
3. Creates and outputs a full granular dataset, at the complaint-officer level, called 'CleanComplaints_FindingLevel.csv'

In [98]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
from pyprojroot import here

In [99]:
# Import data (downloaded from website linked above as Excel file)

## To-do: use Python's openpyxl library to filter the datasets before importing -> improve speed

cmp = pd.read_excel(here('./data/raw/uof_complaints_chicago.xlsx'), sheet_name='Allegations')
witness = pd.read_excel(here('./data/raw/uof_complaints_chicago.xlsx'), sheet_name='Complaining Witnesses')
officer = pd.read_excel(here('./data/raw/uof_complaints_chicago.xlsx'), sheet_name='Officer Profile')

## 1. Cleaning all datasets

### Cleaning main complaints file

In [100]:
# Filtering the data to only include complaints from 2015 onwards

## There are 8 rows where incident date (what we want to filter on) is missing
cmp['IncidentDate'].isnull().sum()

## Drop cases where incidentdate is missing - these are only 8 observations, and there is no other good way
## to tell when a complaint occured. The start date only refers to the start of the investigation, and this could be
## very different from the actual timing of the complaint.
cmp = cmp.dropna(subset=['IncidentDate'])

## Converting incident date to a date variable
cmp['IncidentDate']= pd.to_datetime(cmp['IncidentDate'])

## Keeping only complaints >= 2015
cmp = cmp[cmp['IncidentDate'] >= '2015-01-01']

In [101]:
## Convert CRID to numeric for merging with other datasets
cmp['CRID'] = pd.to_numeric(cmp['CRID'])

## Keep only useful variables
cmp = cmp[['CRID', 'OfficerID', 'Allegation', 'Finding', 'Outcome', 'Beat', 'IncidentDate', 'StartDate', 'EndDate', 'InvestigatorRank',
           'Latitude', 'Longitude']]


### Cleaning officer and witness tables

In [102]:
# Converting appointment date to correct date format
officer['clean_appdate'] = ''

officer.loc[officer['ApptDate'].notna(), 'clean_appdate'] = pd.to_datetime(officer['ApptDate'], unit='d', origin='1899-12-30')

# Create an ordinal variable for officer rank (1 = police officer, 2 = sargeant, etc.)
# Based this on Ranks section of this Wiki page: https://en.wikipedia.org/wiki/Chicago_Police_Department#Ranks

# Drop officers without a rank (45 officers dropped - this also takes care of officers with other pieces of missing information)
officer.dropna(subset=['Rank'], inplace=True)

l = ['Sergeant Of Police', 'Sergeant Per Arbitra', 'Sgt', 'Sgt Assgn Sec Spec']
regstr = '|'.join(l)
officer.loc[officer['Rank'].str.contains(regstr), 'rank_order'] = 2

l = ['Lieutenant Of Police', 'Lt']
regstr = '|'.join(l)
officer.loc[officer['Rank'].str.contains(regstr), 'rank_order'] = 3

officer.loc[officer['Rank'] == 'Captain Of Police', 'rank_order'] = 4

l = ['Commander', 'Cmdr']
regstr = '|'.join(l)
officer.loc[officer['Rank'].str.contains(regstr), 'rank_order'] = 5

officer.loc[officer['Rank'] == 'Dep Chief', 'rank_order'] = 6

officer.loc[officer['Rank'] == 'Chief', 'rank_order'] = 7

l = ['Assistant Superintendent', 'Asst Deputy Sup', 'Deputy Supt.', 'First Deputy Supt.', 'Supt Of Police']
regstr = '|'.join(l)
officer.loc[officer['Rank'].str.contains(regstr), 'rank_order'] = 8

# Put the rest of the officers as rank one
officer['rank_order'] = officer['rank_order'].fillna(1)

# Keeping only relevant info from the table
officer = officer[['OfficerID', 'Gender', 'Race', 'clean_appdate', 'rank_order', 'Age']]

# Renaming columns to keep it ready for the merge
officer.rename(columns={'Gender':'officer_gender', 'Race': 'officer_race', 'rank_order': 'officer_rank_order', 'Age': 'officer_age',
                      'clean_appdate': 'officer_appt_date'}, inplace=True)

# Printing to check
officer.head()

,OfficerID,officer_gender,officer_race,officer_appt_date,officer_rank_order,officer_age
0,1,M,White,2005-09-26 00:00:00,2.0,50
1,2,F,Hispanic,2005-09-26 00:00:00,1.0,41
2,4,M,White,1969-01-06 00:00:00,1.0,79
3,6,M,White,1994-12-05 00:00:00,1.0,53
4,7,M,White,1995-12-04 00:00:00,1.0,49


In [103]:
## Dropping witness age because it's missing for all
witness.drop(columns=['Age'], inplace=True)

## Renaming columns in witness table before merge

witness.rename(columns={'Gender': 'witness_gender', 'Race': 'witness_race'}, inplace = True)

witness.head()

,CRID,witness_gender,witness_race
0,1000002,M,Hispanic
1,1000002,M,Hispanic
2,1000004,M,Hispanic
3,1000004,M,Hispanic
4,1000006,M,Black


### Combine all data into one merged dataset

In [104]:
# Merge with the main table
cmp_full = cmp.merge(officer, how='left', on=['OfficerID'])

cmp_full = cmp_full.merge(witness, how='left', on=['CRID'])

# Drop full duplicates (unsure why these are created - maybe because of duplicates in officer ID table?)
cmp_full.drop_duplicates(inplace=True)

print(cmp_full.columns)
cmp_full.head()

Index(['CRID', 'OfficerID', 'Allegation', 'Finding', 'Outcome', 'Beat',
       'IncidentDate', 'StartDate', 'EndDate', 'InvestigatorRank', 'Latitude',
       'Longitude', 'officer_gender', 'officer_race', 'officer_appt_date',
       'officer_rank_order', 'officer_age', 'witness_gender', 'witness_race'],
      dtype='object')


,CRID,OfficerID,Allegation,Finding,Outcome,Beat,IncidentDate,StartDate,EndDate,InvestigatorRank,Latitude,Longitude,officer_gender,officer_race,officer_appt_date,officer_rank_order,officer_age,witness_gender,witness_race
0,1073214,7936,Excessive Force / On Duty - No Injury,Not Sustained,No Action Taken,313.0,2015-01-03,2015-01-03,2015-04-16,NaN,41.781068,-87.605533,F,Black,2012-12-14 00:00:00,1.0,46.0,M,Black
2,1073237,17217,Excessive Force / On Duty - No Injury,Unfounded,No Action Taken,1112.0,2015-01-03,2015-01-05,2015-09-09,NaN,41.898191,-87.720292,M,Hispanic,2003-08-25 00:00:00,1.0,42.0,M,White
4,1073237,24563,Excessive Force / On Duty - No Injury,Unfounded,No Action Taken,1112.0,2015-01-03,2015-01-05,2015-09-09,NaN,41.898191,-87.720292,M,Hispanic,2003-09-29 00:00:00,1.0,52.0,M,White
6,1073267,23783,Excessive Force / On Duty - No Injury,Not Sustained,No Action Taken,1112.0,2015-01-07,2015-01-07,2015-04-23,NaN,41.895415,-87.719968,M,Hispanic,1994-09-06 00:00:00,1.0,65.0,F,Black
8,1073323,21448,Excessive Force / On Duty - Injury,Unfounded,No Action Taken,932.0,2015-01-03,2015-01-12,2015-06-18,NaN,41.794156,-87.672792,M,Hispanic,1994-09-06 00:00:00,1.0,54.0,M,Black


## 2. Create a basic dataset with just # complaints per tract

In [105]:
# Drop all irrelevant columns
basic_df = cmp_full[['CRID', 'Latitude', 'Longitude']]

# Drop duplicates in CRID - in the imported data, there is a row for every officer linked with the complaint
# For now, we are only interested in the total # of unique complaints
basic_df.drop_duplicates(subset=['CRID'], inplace=True)

# Checking the quality/missingness of lat-long data
print(basic_df.isnull().sum()) 

# There are 22 missing lat-long values. We cannot merge these with the appropriate census tract,
# so dropping these. 
# To-do code: check to see if address information is present for these missing lat-longs, and whether they can be geo-coded
basic_df = basic_df.dropna()

CRID          0
Latitude     22
Longitude    22
dtype: int64


<ipython-input-105-0a832123fb98>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basic_df.drop_duplicates(subset=['CRID'], inplace=True)


In [106]:
# Convert the dataframe to a GeoDataFrame (required for spatial merge with census tracts polygons)
basic_df = gpd.GeoDataFrame(basic_df, geometry=gpd.points_from_xy(basic_df.Longitude, basic_df.Latitude), crs='epsg:4326')

# Importing the census tracts data and converting it to a GeoDataFrame
acs = pd.read_csv(here('./data/CleanACSFile.csv'))

# Keeping only relevant info from acs file for the spatial merge
acs = acs[['geo_id', 'geometry']]

acs['geometry'] = acs['geometry'].apply(wkt.loads)
acs = gpd.GeoDataFrame(acs, crs='epsg:4326')

# Doing the spatial merge to assign a tract ID to every complaint
basic_df = gpd.sjoin(basic_df, acs[['geo_id', 'geometry']], how='left')

basic_df.head()

,CRID,Latitude,Longitude,geometry,index_right,geo_id
0,1073214,41.781068,-87.605533,POINT (-87.60553 41.78107),435.0,1400000US17031420400
2,1073237,41.898191,-87.720292,POINT (-87.72029 41.89819),279.0,1400000US17031231200
6,1073267,41.895415,-87.719968,POINT (-87.71997 41.89542),280.0,1400000US17031231500
8,1073323,41.794156,-87.672792,POINT (-87.67279 41.79416),557.0,1400000US17031611700
10,1073326,41.750469,-87.635831,POINT (-87.63583 41.75047),783.0,1400000US17031842400


In [107]:
# Checking how many observations did not match to a tract ID
print(basic_df.isnull().sum()) # 20 observations did not find a census tract match

basic_df[basic_df['geo_id'].isnull()]

# Note: manuall checking these lat-longs, it appears that these are valid-latlongs, but fall
# outside of the City of Chicago boundaries. E.g., CRID 1074942 was from Oak Lawn, which falls to the west of the boundaries

# Since these are outside of the focus area of the study (City of Chicago), we are dropping these 20 observations
basic_df.dropna(inplace=True)

CRID            0
Latitude        0
Longitude       0
geometry        0
index_right    20
geo_id         20
dtype: int64


In [108]:
# Grouping by the census ID to create a count of crime IDs for each tract
counts = basic_df[['CRID', 'geo_id']].groupby('geo_id').count()

# Merging these counts back with the full dataset of census tract IDs
merged = acs.merge(counts, how='left', on=['geo_id'])

# Replacing missing crime ID count values with 0 (i.e. missing means there were 0 complaints found in that tract)
merged['CRID'] = merged['CRID'].fillna(0)

# Renaming column
merged.rename(columns={'CRID':'complaint_count'}, inplace=True)

merged.head()

,geo_id,geometry,complaint_count
0,1400000US17031010100,"MULTIPOLYGON (((-87.67720 42.02294, -87.67007 ...",0.0
1,1400000US17031010201,"MULTIPOLYGON (((-87.68465 42.01949, -87.68045 ...",0.0
2,1400000US17031010202,"MULTIPOLYGON (((-87.67685 42.01941, -87.67339 ...",1.0
3,1400000US17031010300,"MULTIPOLYGON (((-87.67133 42.01937, -87.66950 ...",0.0
4,1400000US17031010400,"MULTIPOLYGON (((-87.66345 42.01283, -87.66133 ...",0.0


In [109]:
# Export .CSV file to be used in other scripts
merged[['geo_id', 'complaint_count']].to_csv(here('./data/CleanComplaints.csv'),
                                            encoding='utf-8', index=False)

## 3. Create and explort a more detailed finding-level dataset 

In [110]:

cmp_full.head()

'''
To-dos:
- process allegation data into categories (group them into types)
- create officer vintage variable
- Extract incident year 
'''

# Spatially merging tract-id on these complaints so we can do merges with more features later
cmp_full = cmp_full.merge(basic_df, how='left', on=['CRID'])

cmp_full.columns

# Create variable on how long the officer had been employed for at the time of the incident
#cmp_full['officer_vintage'] = cmp_full['IncidentDate'] - cmp_full['officer_appt_date'] 

#cmp_full.head()

#type(cmp_full['officer_appt_date'][0])

## Exporting this full dataset to be read into Question 2
cmp_full.to_csv(here('./data/CleanComplaints_FindingLevel.csv'), encoding='utf-8', index=False)

# References

Code sourced:
- https://stackoverflow.com/questions/38454403/convert-excel-style-date-with-pandas
- https://www.geeksforgeeks.org/python-convert-excel-serial-date-to-datetime/


In [111]:
### Archive code that could be used/repurposed later

# Seeing the proportion of complaints that were sustained

#cmp.drop_duplicates(subset=['CRID'], inplace=True)

# cmp[['CRID', 'Finding']].groupby('Finding').count()

## Dropping irrelevant columns
#officers.drop(['OfficerFirst', 'OfficerLast'])

## Renaming variables before merging on CRID
#officers.columns = ['CRID', '']

## Dropping irrelevant columns
#complaints.drop(['OfficeFirst', 'OfficerLast', 'AllegationCode', 'RecommendedFinding', 'RecommendedOutcome',
#                'FinalFinding', 'FinalOutcome', ''], axis=1)

## Drop irrelevant columns, merge the info across the three datasets
#print(complaints.info(), '\n', comp_witness.info(), '\n', officers.info())

#cmp.head()

#cmp[['Beat', 'Location', 'City']].head(100)

#cmp['Diff'] = np.where(cmp['RecommendedFinding'] == cmp['FinalFinding'], 1, 0)

#cmp['Diff'].mean()

## The unique ID here is CRID and officerID
## Drop the unnecessary columns, merge the good columns from the other datasets, then describe the missingness and uniqueness 

#cmp.describe()
#cmp.info()

#cmp.nunique(axis=0)

#cmp[cmp['CRID'] == '1090030']

#cmp['uid'] = cmp['CRID'] + (cmp['OfficerID']).astype(str)

#cmp.nunique(axis=0)


#dups = cmp[cmp.duplicated(['CRID'])].sort_values('CRID')

#dups.head(100)

